### In this problem, we are given a dataset that contains goal scoring statistics from Cristiano Ronaldo's career during the period 1996-2016.
### Our aim is to find the relationship between different variables and the target and also to predict whether a given shot will be a GOAL or NOT.  

In [28]:
import pandas as pd
import numpy as np
import warnings
import random
warnings.simplefilter(action='ignore')
df=pd.read_csv(r'C:\Machine learning\other datasets\ronaldo_data.csv')
df.describe()

,Unnamed: 0,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,remaining_sec,distance_of_shot,is_goal,shot_id_number,match_id,team_id,remaining_min.1,power_of_shot.1,knockout_match.1,remaining_sec.1,distance_of_shot.1
count,30697.000000,29134.000000,29236.000000,29157.000000,29135.000000,29211.000000,29180.000000,29103.000000,29130.000000,24429.000000,29134.000000,3.069700e+04,3.069700e+04,29162.000000,29158.000000,29204.000000,29158.000000,29129.000000
mean,15348.000000,249.576028,7.383876,91.126933,4.883233,2.519359,0.145956,28.329382,33.448884,0.445331,15368.581039,2.476407e+07,1.610613e+09,18.204615,15.994109,16.599402,39.027303,38.801852
std,8861.604943,150.186019,110.263049,87.676395,3.452533,1.153976,0.353068,17.470663,9.369656,0.497013,8866.520772,7.755175e+06,0.000000e+00,29.416973,29.676815,35.172016,29.835284,18.787711
min,0.000000,2.000000,-250.000000,-44.000000,0.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,2.000001e+07,1.610613e+09,0.000000,1.000000,0.000000,0.000000,9.400000
25%,7674.000000,111.000000,-68.000000,4.000000,2.000000,1.000000,0.000000,13.000000,25.000000,0.000000,7693.250000,2.050008e+07,1.610613e+09,3.000000,2.000000,0.000000,17.000000,26.000000
50%,15348.000000,254.000000,0.000000,74.000000,5.000000,3.000000,0.000000,28.000000,35.000000,0.000000,15378.500000,2.090035e+07,1.610613e+09,6.000000,3.000000,0.000000,35.000000,36.000000
75%,23022.000000,369.000000,95.000000,160.000000,8.000000,3.000000,0.000000,43.000000,41.000000,1.000000,23055.750000,2.960047e+07,1.610613e+09,11.000000,4.000000,1.000000,52.000000,44.000000
max,30696.000000,659.000000,248.000000,791.000000,11.000000,7.000000,1.000000,59.000000,99.000000,1.000000,30697.000000,4.990009e+07,1.610613e+09,128.761600,118.360000,141.352320,144.785600,115.728000


In [29]:

#X_cols=['location_x','location_y','power_of_shot','match_id','area_of_shot','game_season','distance_of_shot','remaining_min','lat/lng','shot_basics','range_of_shot','type_of_shot']
X_cols=['location_x','location_y','area_of_shot','distance_of_shot','remaining_min','shot_basics','range_of_shot','type_of_shot']
cols_to_encode=['shot_basics','range_of_shot','type_of_shot','area_of_shot']
cols_to_scale_min=['distance_of_shot']
cols_to_scale_sc=['location_x','location_y']

X=df.loc[:,X_cols]

#preprocessing

# filling up missing values in area_of_shot
X.loc[(X.shot_basics == 'Goal Area')|(X.shot_basics == 'Goal Line'),'area_of_shot']=\
X.loc[(X.shot_basics == 'Goal Area')|(X.shot_basics == 'Goal Line'),'area_of_shot'].fillna(value='Center(C)')

X.loc[X.shot_basics == 'Left Corner','area_of_shot']=\
X.loc[X.shot_basics == 'Left Corner','area_of_shot'].fillna(value='Left Side(L)')

X.loc[X.shot_basics == 'Right Corner','area_of_shot']=\
X.loc[X.shot_basics == 'Right Corner','area_of_shot'].fillna(value='Right Side(R)')

p_total=df.loc[df.shot_basics == 'Mid Range','area_of_shot'].value_counts().sum()
p=df.loc[df.shot_basics == 'Mid Range','area_of_shot'].value_counts().to_list()/p_total
X.loc[X.shot_basics == 'Mid Range','area_of_shot']=\
X.loc[X.shot_basics == 'Mid Range','area_of_shot'].apply(lambda x:x if type(x) == str else\
                                                          np.random.choice(['Center(C)','Left Side(L)','Right Side Center(RC)',\
                                                                               'Left Side Center(LC)','Right Side(R)'],1,p=p)[0])

X.loc[X.shot_basics == 'Mid Ground Line','area_of_shot']=\
X.loc[X.shot_basics == 'Mid Ground Line','area_of_shot'].fillna(value='Mid Ground(MG)')

p_total=df.loc[df.shot_basics == 'Penalty Spot','area_of_shot'].value_counts().sum()
p=df.loc[df.shot_basics == 'Penalty Spot','area_of_shot'].value_counts().to_list()/p_total
X.loc[X.shot_basics == 'Penalty Spot','area_of_shot']=\
X.loc[X.shot_basics == 'Penalty Spot','area_of_shot'].apply(lambda x: x if type(x) == str else\
                                                             np.random.choice(['Center(C)','Right Side Center(RC)',\
                                                                               'Left Side Center(LC)','Mid Ground(MG)'],1,p=p)[0])

# replacing empty values for location coordinates with 0
X['location_x'].fillna(value=0,inplace=True)
X['location_y'].fillna(value=0,inplace=True)

# filling type of shot and remaining minutes with respective mode values
#X['power_of_shot'].fillna(value=X['power_of_shot'].mean(),inplace=True)
X['type_of_shot'].fillna(value=X['type_of_shot'].mode()[0],inplace=True)
X['remaining_min'].fillna(value=X['remaining_min'].mode()[0],inplace=True)

# filling up missing values in distance_of_shot
p=0
p=X.loc[X.range_of_shot =='Less Than 8 ft.','distance_of_shot'].value_counts().head(8).sum()
p_val=X.loc[X.range_of_shot =='Less Than 8 ft.','distance_of_shot'].value_counts().head(8).to_list()/p
X.loc[X.range_of_shot =='Less Than 8 ft.','distance_of_shot']=\
X.loc[X.range_of_shot =='Less Than 8 ft.','distance_of_shot'].apply(lambda x:x if type(x) is int\
else np.random.choice([20,27,26,21,22,25,23,24],1,p=p_val)[0])

p=X.loc[X.range_of_shot =='8-16 ft.','distance_of_shot'].value_counts().head(8).sum()
p_val=X.loc[X.range_of_shot =='8-16 ft.','distance_of_shot'].value_counts().head(8).to_list()/p
X.loc[X.range_of_shot =='8-16 ft.','distance_of_shot']=\
X.loc[X.range_of_shot =='8-16 ft.','distance_of_shot'].apply(lambda x:x if type(x) is int\
else np.random.choice([35,34,33,32,30,31,28,29],1,p=p_val)[0])

p=X.loc[X.range_of_shot =='16-24 ft.','distance_of_shot'].value_counts().head(8).sum()
p_val=X.loc[X.range_of_shot =='16-24 ft.','distance_of_shot'].value_counts().head(8).to_list()/p
X.loc[X.range_of_shot =='16-24 ft.','distance_of_shot']=\
X.loc[X.range_of_shot =='16-24 ft.','distance_of_shot'].apply(lambda x:x if type(x) is int\
else np.random.choice([37,36,38,39,40,41,42,43],1,p=p_val)[0])

p=X.loc[X.range_of_shot =='24+ ft.','distance_of_shot'].value_counts().head(6).sum()
p_val=X.loc[X.range_of_shot =='24+ ft.','distance_of_shot'].value_counts().head(6).to_list()/p
X.loc[X.range_of_shot =='24+ ft.','distance_of_shot']=\
X.loc[X.range_of_shot =='24+ ft.','distance_of_shot'].apply(lambda x:x if type(x) is int\
else np.random.choice([45,44,46,47,43,48],1,p=p_val)[0])

X.loc[X.range_of_shot =='Back Court Shot','distance_of_shot']=\
X.loc[X.range_of_shot =='Back Court Shot','distance_of_shot'].fillna(random.choice([78,94,82,87,85]))

# conditional filling for range_of_shot
X.loc[(X.shot_basics == 'Goal Area') | (X.shot_basics == 'Goal Line'), 'range_of_shot'] =\
X.loc[(X.shot_basics == 'Goal Area') | (X.shot_basics == 'Goal Line'), 'range_of_shot'].fillna(value='Less Than 8 ft.')

X.loc[(X.shot_basics == 'Mid Range'), 'range_of_shot'] =\
X.loc[(X.shot_basics == 'Mid Range'), 'range_of_shot'].fillna(value=random.choice(['8-16 ft.','16-24 ft.']))

X.loc[(X.shot_basics == 'Penalty Spot')|(X.shot_basics == 'Left Corner')|(X.shot_basics == 'Right Corner'), 'range_of_shot'] =\
X.loc[(X.shot_basics == 'Penalty Spot')|(X.shot_basics == 'Left Corner')|(X.shot_basics == 'Right Corner'), 'range_of_shot'].\
fillna(value='24+ ft.')

X.loc[(X.shot_basics == 'Mid Ground Line'), 'range_of_shot'] =\
X.loc[(X.shot_basics == 'Mid Ground Line'), 'range_of_shot'].fillna(value='Back Court Shot')

# filling empty values for shot basics
X.loc[X.range_of_shot=='Less Than 8 ft.','shot_basics']=X.loc[X.range_of_shot=='Less Than 8 ft.','shot_basics']\
.fillna(value=random.choice(['Goal Area','Goal Line']))

X.loc[X.range_of_shot=='24+ ft.','shot_basics']=X.loc[X.range_of_shot=='24+ ft.','shot_basics']\
.fillna(value=random.choice(['Penalty Spot','Left Corner','Right Corner']))

X.loc[(X.range_of_shot=='8-16 ft.')|(X.range_of_shot=='16-24 ft.'),'shot_basics']\
=X.loc[(X.range_of_shot=='8-16 ft.')|(X.range_of_shot=='16-24 ft.'),'shot_basics']\
.fillna(value='Mid Range')

# using forward fill in case there are still any missing values.
X['shot_basics'].fillna(method='ffill',inplace=True)
X['range_of_shot'].fillna(method='ffill',inplace=True)
X['distance_of_shot'].fillna(method='ffill',inplace=True)
X['area_of_shot'].fillna(method='ffill',inplace=True)

# end of preprocessing


# setting target variable y and dropping rows where y has missing values
y=df.loc[:,'is_goal']
y=y.dropna()
dropped_rows=df[~df.index.isin(y.index)].values[:,0]
X=X.drop(dropped_rows)
X.reset_index(drop=True,inplace=True)

#X.isna().sum()

# encoding the categorical variables
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,StandardScaler
for i in cols_to_encode:
    le=LabelEncoder()
    X[i]=le.fit_transform(X[i])

from sklearn.feature_selection import mutual_info_classif
def mutual_info(X,y):
    mi=mutual_info_classif(X,y)
    mi=pd.Series(mi,name='miscore',index=X.columns)
    mi=mi.sort_values(ascending=False)
    return mi
mi_sc=mutual_info(X,y)
print(mi_sc)


distance_of_shot    0.021937
shot_basics         0.020924
range_of_shot       0.015703
area_of_shot        0.014678
location_y          0.013315
location_x          0.011458
type_of_shot        0.010765
remaining_min       0.003223
Name: miscore, dtype: float64


In [30]:

# splitting of data to train and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=21)


# standardize all values in X before training to give equal importance to all features
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


#disc_features=[X.dtypes == int]


#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
#from sklearn.svm import SVC
#model=RandomForestClassifier(n_estimators=50,criterion='entropy',random_state=0)
#model=LogisticRegression(random_state=1)
model=DecisionTreeClassifier(criterion='entropy',splitter='best',min_samples_leaf=4,max_features=6,max_leaf_nodes=5)
#model=SVC(kernel='rbf',C=2,gamma=0.01,random_state=2)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

from sklearn.metrics import accuracy_score,confusion_matrix
print(confusion_matrix(y_test,y_pred),end='\n')
print(accuracy_score(y_test,y_pred))


[[2397  351]
 [1511  627]]
0.6189111747851003


In [44]:
from sklearn.model_selection import cross_val_score
cv=cross_val_score(estimator=model,X=X_train,y=y_train,cv=10)
print(cv)
acc=cv.mean()
print(acc)

[0.60767263 0.61074169 0.60869565 0.59928352 0.60286592 0.59109519
 0.59928352 0.6171955  0.61003071 0.61566018]
0.6062524508713192
